# **Feature detection and overlaying**

In this section we will create overlaying function and our main function for feature retrieval and then overlaying the props over those identified features.

Download the rar containing props from [here](https://drive.google.com/file/d/1-fPGb4NBhEBN9hbC-rYMlea8vta6Dlc7/view?usp=sharing) and extract in your working directory

**Haar Cascade Classifiers**

OpenCV already contains many pre-trained classifiers for face, eyes, smile etc. XML files containing the wights of the model are stored in opencv/data/haarcascades/ folder and can be used for feature detection quickly and easily.

OpenCV's algorithm is uses the following Haar-like features which are the input to the basic classifiers:

<img src="https://www.bogotobogo.com/python/OpenCV_Python/images/FaceDetection/HaarFeatures.png" align="center"/>

In short here is how feature detection works, instead of applying all the 6000 features on a window, group the features into different stages of classifiers and apply one-by-one. 

If a window fails the first stage, discard it. We don't consider remaining features on it. If it passes, apply the second stage of features and continue the process. The window which passes all stages is a face region.

For example:

<img src="https://www.bogotobogo.com/python/OpenCV_Python/images/FaceDetection/stages.png" align="center"/>

Complete discussion about the Haar Cascade Classifiers is beyond the scope of our current discussion but checkout *Further Readings* section for more details.

**Alpha Blending for overlaying**

Alpha blending is the process of overlaying a foreground image with transparency over a background Image. The transparent image is generally a PNG image which consists of four channels (RGBA). The fourth channel is the alpha channel which holds the transparency magnitude.
Here background image will be our source image and foreground image will be thug life props.

At every pixel of the image, we blend the background and foreground image color(F) and background color (B) using the alpha mask.

Explained clearly [here](https://pytech-solution.blogspot.com/2017/07/alphablending.html)

In [ ]:
def transparentOverlay(src, overlay, pos=(0, 0), scale=1):
    '''
    This function is used to blend the props over the identified features
    Takes four arguments.
    src : this is the original image (frames – as we loop over the frames from the video stream)
    overlay : this refers to the overlay images (shades and cigar)
    position and scaling factor
    '''

    # For changing the dimension of the image
    overlay = cv2.resize(overlay, (0, 0), fx=scale, fy=scale)
    
    # Size of foreground
    h, w, _ = overlay.shape
    # Size of background Image
    rows, cols, _ = src.shape

    y, x = pos[0], pos[1]  # Position of foreground/overlay image
 
    # Loop over all pixels and apply the blending equation
    for i in range(h):
        for j in range(w):
            # Check the bounding conditions
            if x + i >= rows or y + j >= cols:
                continue
            # Read the alpha channel
            alpha = float(overlay[i][j][3] / 255.0)
            src[x + i][y + j] = alpha * overlay[i][j][:3] + (1 - alpha) * src[x + i][y + j]
    
    return src

In [ ]:
def ThugLife(src, specs, cigar):
    '''
    A void function that takes 3 arguments and transforms the input image to image with thug life filter applied
    src: string that conatins path to image file
    specs: path to png specs prop as in thug life playground
    cigar: path to png cigar prop as in thug life playground
    '''

    # Reading the images
    img = cv2.imread(src, -1)
    specs = cv2.imread(specs, -1)
    cigar = cv2.imread(cigar,-1)

    cv2.imshow('Original',img)

    # Converting coloured images to black and white and hence reducing the computation by a factor of 3 [Coloured images: 3 channels, Gray images: 1 channel]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Loads the weights of cascade classifier for faces from the file (You don't need to download the xml files ! They are present in the library)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Used to find faces from the image. If eyes are found, it returns the positions of detected faces as Rect(x,y,w,h)
    faces = face_cascade.detectMultiScale(img, 1.2, 5, 0, (120, 120), (350, 350))

    #  Iterating through the faces and creating Region of Interest (RoI) and then overlaying
    for (x, y, w, h) in faces:
        if h > 0 and w > 0:
          # Calculating the region where specs are to be overlayed
          glass_ymin = int(y + 1.5 * h / 5)
          glass_ymax = int(y + 2.5 * h / 5)
          h_glass = glass_ymax - glass_ymin

          # Calculating the region where cigar is to be overlayed
          cigar_ymin = int(y + 4 * h / 6)
          cigar_ymax = int(y + 5.5 * h / 6)
          h_cigar = cigar_ymax - cigar_ymin

          # Note that we are going to overlay over the color image but the RoI was created over Gray image
          glass_roi = img[glass_ymin:glass_ymax, x:x+w]
          cigar_roi = img[cigar_ymin:cigar_ymax, x:x+w]

          # Resizing the props according to INTER_CUBIC interpolation (Best for resizing, but slow)
          specs = cv2.resize(specs, (w, h_glass),interpolation=cv2.INTER_CUBIC)
          cigar= cv2.resize(cigar, (w, h_cigar),interpolation=cv2.INTER_CUBIC)

          # Applying the overlays, calling the helper function
          transparentOverlay(glass_roi,specs)
          transparentOverlay(cigar_roi,cigar,(int(w/2),int(h_cigar/2)))

    cv2.imshow('ThugLife', img)

    # Writing the file and storing in current directory
    cv2.imwrite('output.jpg', img)

    # Image will be displayed infinitely until any keypress
    cv2.waitKey(0)

# **Testing the functions**

In this section we will first load the props already provided with the folder and provide the path for source image and check the functions we created.

Two windows will open displaying the original image and the output image.

In [ ]:
# Path to props
specs = '.img/specs.png'
cigar = '.img/cigar.png'

In [ ]:
# Add path to input image
img = "image.jpeg"

In [ ]:
ThugLife(img, specs, cigar)
# Or you can also do this, Now whenever space will be clicked your image will be captured and then transformed
ThugLife(CaptureImage(), specs, cigar)

**Expected Output**

<img src="https://raw.githubusercontent.com/viveksb007/ThugLifeImgCreator/master/sshots/s3.png" align="center"/>

# **Further Reading**

Congrats !! Now you have the recipe for transforming the images as done in the Thug Life playground of our website.

Also read and continue building more cool stuff -

*  [Rapid Object Detection using a Boosted Cascade of Simple Features](https://www.researchgate.net/publication/3940582_Rapid_Object_Detection_using_a_Boosted_Cascade_of_Simple_Features)

* [Image filtering and processing using Adversarial Networks](http://www.jetir.org/papers/JETIR1703019.pdf)